### Backtesting the strategy

### Importing the required libraries.

In [25]:
import pandas as pd
import pandas_ta as ta

### Load your dataset


In [26]:
file_path = '/backtestingdata.xlsx'
data = pd.read_excel(file_path)
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

### Calculate SMA, Bollinger Bands, and RSI


In [27]:
data['SMA'] = ta.sma(data['close'], length=7)
bbands = ta.bbands(data['close'], length=7, std=1.4)
data['RSI'] = ta.rsi(data['close'], length=14)
data = pd.concat([data, bbands], axis=1)

### Initialize columns for signals and positions

In [28]:
data['signal'] = 0  # Buy=1, Sell=-1, Hold=0
data['position'] = 0

### Define RSI thresholds

In [29]:
rsi_lower_threshold = 30
rsi_upper_threshold = 70

### Generate Signals

In [ ]:
for i in range(1, len(data)):
    # Buy signals
    if (data['close'][i] > data['SMA'][i] or data['close'][i] > data['BBL_7_1.4'][i]) and data['RSI'][i] < rsi_lower_threshold:
        data['signal'][i] = 1

    # Sell signals
    elif (data['close'][i] < data['SMA'][i] or data['close'][i] < data['BBU_7_1.4'][i]) and data['RSI'][i] > rsi_upper_threshold:
        data['signal'][i] = -1

    # Hold
    else:
        data['signal'][i] = 0

### Determine positions (1 for held, 0 for not held)

In [31]:
data['position'] = data['signal'].replace(to_replace=0, method='ffill')

### Backtest performance (assuming 1 share traded per signal)

In [32]:
data['daily_return'] = data['close'].pct_change()
data['strategy_return'] = data['daily_return'] * data['position'].shift()

### Calculate cumulative returns

In [33]:
data['cumulative_market_return'] = (1 + data['daily_return']).cumprod()
data['cumulative_strategy_return'] = (1 + data['strategy_return']).cumprod()

### Calculate total profit/loss

In [34]:
initial_investment = 100000  # Example: 100,000 currency units
market_profit_loss = (data['cumulative_market_return'].iloc[-1] - 1) * initial_investment
strategy_profit_loss = (data['cumulative_strategy_return'].iloc[-1] - 1) * initial_investment

### Display results

In [35]:
print(f"Market Profit/Loss: {market_profit_loss:.2f}")
print(f"Strategy Profit/Loss: {strategy_profit_loss:.2f}")

Market Profit/Loss: 1657.96
Strategy Profit/Loss: 1521.13
